In [1]:
# 시뮬레이터 회귀 데이터 생성
import numpy
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=500, n_features=2, n_informative=2, noise=5, random_state=0)
# 데이터 세트 크기 출력
print("Number of Examples in the Dataset : ", X.shape[0])
print("Number of Features for each example : ", X.shape[1])
# 결과 범위 출력
print("Output Range = [%f, %f]" % (min(y), max(y)))

Number of Examples in the Dataset :  500
Number of Features for each example :  2
Output Range = [-288.225754, 271.270135]


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation

# 케라스 모델을 리턴하는 함수 생성
def build_model_1():
    # 케라스 모델 구성
    model = Sequential()
    model.add(Dense(4, activation='relu', input_dim=2))
    model.add(Dense(1))
    # 모델 컴파일
    model.compile(loss='mean_squared_error', optimizer='adam')
    # 모델 리턴
    return model

def build_model_2():
    # 케라스 모델 구성
    model = Sequential()
    model.add(Dense(8, activation='relu', input_dim=2))
    model.add(Dense(1))
    # 모델 컴파일
    model.compile(loss='mean_squared_error', optimizer='adam')
    # 모델 리턴
    return model

def build_model_3():
    # 케라스 모델 구성
    model = Sequential()
    model.add(Dense(16, activation='relu', input_dim=2))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    # 모델 컴파일
    model.compile(loss='mean_squared_error', optimizer='adam')
    # 모델 리턴
    return model

Using TensorFlow backend.


In [3]:
# 랜덤 넘버 생성자 시드 설정
seed = 1
numpy.random.seed(seed)
# 각 모델에 대한 교차 검증 수행
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
#
results = []
models = [build_model_1, build_model_2, build_model_3]
# 3개 모델을 대상으로 반복
for m in range(len(models)):
    model = KerasRegressor(build_fn= models[m], epochs=300, batch_size=10, verbose=0)
    kf = KFold(n_splits=3)
    result = cross_val_score(model, X, y, cv=kf)
    #
    results.append(result)

In [4]:
# print the cross validation scores
print("Cross Validation Loss for Model 1 =", abs(results[0].mean()))
print("Cross Validation Loss for Model 2 =", abs(results[1].mean()))
print("Cross Validation Loss for Model 3 =", abs(results[2].mean()))

Cross Validation Loss for Model 1 = 4039.5688106014973
Cross Validation Loss for Model 2 = 189.20642704206054
Cross Validation Loss for Model 3 = 26.44854514460037


In [5]:
# build_model_3 이 가장 좋으므로, 여기에서 이제 최적의 epocs 와 batch_size 를 구해보자
# 결과를 재현할 수 있도록 랜덤 넘버 생성자 시드 설정
seed = 1
numpy.random.seed(seed)
results = []
epochs = [300, 350]
batches = [10, 15]
# epochs 와 batches 조합 반복 수행
for e in range(len(epochs)):
    for b in range(len(batches)):
        model = KerasRegressor(build_fn= build_model_3, epochs=epochs[e], batch_size=batches[b], verbose=0)
        kf = KFold(n_splits=3)
        result = cross_val_score(model, X, y, cv=kf)
        #
        results.append(result)

In [6]:
c = 0
for e in range(len(epochs)):
    for b in range(len(batches)):
        print("batch_size =", batches[b],", epochs =", epochs[e], ", Test Loss =", abs(results[c].mean()))
        c += 1

batch_size = 10 , epochs = 300 , Test Loss = 26.178123007799282
batch_size = 15 , epochs = 300 , Test Loss = 28.67553945424004
batch_size = 10 , epochs = 350 , Test Loss = 25.79510752105456
batch_size = 15 , epochs = 350 , Test Loss = 27.63913581366893


In [8]:
# 적합한 batch_size 와 epochs 를 구했으므로, 이제 활성화 함수와 최적화 옵션을 구해보자
def build_model_3(activation='relu', optimizer='adam'):
    # 케라스 모델 구성
    model = Sequential()
    model.add(Dense(16, activation=activation, input_dim=2))
    model.add(Dense(8, activation=activation))
    model.add(Dense(1))
    # 모델 컴파일
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    # 모델 리턴
    return model

seed = 1
numpy.random.seed(seed)
results = []
activations = ['relu', 'tanh']
optimizers = ['sgd', 'adam', 'rmsprop']

# activations 와 optimizers 조합 반복 수행
for o in range(len(optimizers)):
    for a in range(len(activations)):
        optimizer = optimizers[o]
        activation = activations[a]
        model = KerasRegressor(build_fn= build_model_3, epochs=350, batch_size=10, verbose=0)
        kf = KFold(n_splits=3)
        result = cross_val_score(model, X, y, cv=kf)
        #
        results.append(result)

In [9]:
c = 0
for o in range(len(optimizers)):
    for a in range(len(activations)):
        print("activation = ", activations[a],", optimizer = ", optimizers[o], ", Test Loss = ", abs(results[c].mean()))
        c += 1

activation =  relu , optimizer =  sgd , Test Loss =  26.317021007561692
activation =  tanh , optimizer =  sgd , Test Loss =  25.159642975593467
activation =  relu , optimizer =  adam , Test Loss =  26.95529824909995
activation =  tanh , optimizer =  adam , Test Loss =  26.243858001851546
activation =  relu , optimizer =  rmsprop , Test Loss =  27.115181424322316
activation =  tanh , optimizer =  rmsprop , Test Loss =  25.0981845816266
